In [1]:
import pandas
import matplotlib
from bokeh.plotting import figure, output_notebook, show

In [2]:
%load_ext google.cloud.bigquery
%matplotlib inline

In [3]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/mforde/google-creds/creds.json'
print('creds path: {}'.format(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))

creds path: /home/mforde/google-creds/creds.json


In [192]:
%%bigquery wins
SELECT season, h_win, COUNT(*) AS count
FROM (
    SELECT season, h_points, a_points, (h_points > a_points) AS h_win
    FROM `bigquery-public-data.ncaa_basketball.mbb_games_sr`
    WHERE NOT IS_NAN(h_points) AND NOT IS_NAN(a_points) # some bad data in 2013 season
        AND NOT(neutral_site)
)
GROUP BY season, h_win
ORDER BY season ASC, h_win

In [193]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from math import pi
from bokeh.models import ColumnDataSource

labels = []
for i in range(0, len(wins)):
    title = ''
    if wins.loc[i]['h_win']:
        title = 'Home Wins'
    else:
        title = "Away Wins"
    label = str(wins.loc[i]['season']) + " " + title
    labels.append(label)
    
wins['labels'] = labels

source = ColumnDataSource(data=wins)

p = figure(x_range=wins["labels"], plot_height=350, title="Does Playing At Home Mean Wins?")
, source=source)
p.vbar(x="labels", top="count", width=0.9, source=source)
p.xaxis.major_label_orientation = pi/4

output_notebook()
show(p)

Loading BokehJS ...

In [97]:
%%bigquery numGames
SELECT season, COUNT(DISTINCT game_id) as numberOfGames
FROM `bigquery-public-data.ncaa_basketball.mbb_games_sr`
GROUP BY season
ORDER BY season ASC

In [100]:
source = ColumnDataSource(data=numGames)

p = figure(plot_height=350, title="Does Playing At Home Mean Wins?")
, source=source)
p.vbar(x="season", top="numberOfGames", width=0.9, source=source)
p.xaxis.major_label_orientation = pi/4

output_notebook()
show(p)

Loading BokehJS ...

In [103]:
print(numGames)

   season  numberOfGames
0    2013           5944
1    2014           5932
2    2015           5951
3    2016           5974
4    2017           6004


In [96]:
%%bigquery
SELECT season, COUNT(DISTINCT team_id) as numberOfTeams
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE division_alias = 'D1'
GROUP BY season
# The "D1" alias is back-filled with the current team division, so we can't see historical data on who has been D1/D2 etc.

,season,numberOfTeams
0,2013,351
1,2017,351
2,2015,351
3,2014,351
4,2016,351


In [ ]:
# NU team_id: 93df9b18-e9fc-42a7-bb45-a736c203b4dc

In [109]:
%%bigquery
SELECT season, COUNT(DISTINCT game_id) as gamesPlayed
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE team_id="93df9b18-e9fc-42a7-bb45-a736c203b4dc" AND tournament="NCAA"
GROUP BY season

,season,gamesPlayed
0,2014,1


In [131]:
%%bigquery topTenByNcaaAppearance
SELECT team_id, market, name, COUNT(DISTINCT game_id) as ncaaGamesPlayed
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE tournament="NCAA"
GROUP BY team_id, market, name
ORDER BY ncaaGamesPlayed DESC
LIMIT 10

In [132]:
print(topTenByNcaaAppearance)

                                team_id          market         name  \
0  2267a1f4-68f6-418b-aaf6-2aa0c4b291f1        Kentucky     Wildcats   
1  e9ca48b2-00ba-41c0-a02b-6885a2da1ff1  North Carolina    Tar Heels   
2  2f4d21f8-6d5f-48a5-abca-52a30583871a         Gonzaga     Bulldogs   
3  4383eb6a-7fd8-4ff2-94c5-43c933121e88       Villanova     Wildcats   
4  c7569eae-5b93-4197-b204-6f3a62146b25       Wisconsin      Badgers   
5  fae4855b-1b64-4b40-a632-9ed345e1e952          Kansas     Jayhawks   
6  faeb1160-5d15-4f26-99fc-c441cf21fc7f            Duke  Blue Devils   
7  1da70895-f77f-44ef-b216-d63c02e696eb          Oregon        Ducks   
8  bdc2561d-f603-4fab-a262-f1d2af462277        Michigan   Wolverines   
9  a41d5a05-4c11-4171-a57e-e7a1ea325a6d  Michigan State     Spartans   

   ncaaGamesPlayed  
0               20  
1               19  
2               18  
3               18  
4               17  
5               17  
6               16  
7               15  
8               15

In [191]:
source = ColumnDataSource(data=topTenByNcaaAppearance)

p = figure(x_range=topTenByNcaaAppearance.market.unique(), plot_height=350, title="Top 10 Teams by Number of NCAA appearances (2013-2018)")
, source=source)
p.vbar(x="market", top="ncaaGamesPlayed", width=0.9, source=source)
p.xaxis.major_label_orientation = pi/4

output_notebook()
show(p)

Loading BokehJS ...

In [171]:
%%bigquery topTeamWins
SELECT season, team_id, market, name, COUNT(DISTINCT game_id) as ncaaGamesWon
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE tournament="NCAA" AND 
    (team_id="2267a1f4-68f6-418b-aaf6-2aa0c4b291f1" OR
     team_id="e9ca48b2-00ba-41c0-a02b-6885a2da1ff1" OR
     team_id="2f4d21f8-6d5f-48a5-abca-52a30583871a" OR
     team_id="4383eb6a-7fd8-4ff2-94c5-43c933121e88" OR
     team_id="c7569eae-5b93-4197-b204-6f3a62146b25") # OR
#      team_id="fae4855b-1b64-4b40-a632-9ed345e1e952" OR
#      team_id="faeb1160-5d15-4f26-99fc-c441cf21fc7f" OR
#      team_id="1da70895-f77f-44ef-b216-d63c02e696eb" OR
#      team_id="bdc2561d-f603-4fab-a262-f1d2af462277" OR     
#      team_id="a41d5a05-4c11-4171-a57e-e7a1ea325a6d")
    AND win
GROUP BY season, team_id, market, name
ORDER BY team_id, season ASC

In [172]:
print(topTeamWins)

    season                               team_id          market       name  \
0     2013  2267a1f4-68f6-418b-aaf6-2aa0c4b291f1        Kentucky   Wildcats   
1     2014  2267a1f4-68f6-418b-aaf6-2aa0c4b291f1        Kentucky   Wildcats   
2     2015  2267a1f4-68f6-418b-aaf6-2aa0c4b291f1        Kentucky   Wildcats   
3     2016  2267a1f4-68f6-418b-aaf6-2aa0c4b291f1        Kentucky   Wildcats   
4     2017  2267a1f4-68f6-418b-aaf6-2aa0c4b291f1        Kentucky   Wildcats   
5     2013  2f4d21f8-6d5f-48a5-abca-52a30583871a         Gonzaga   Bulldogs   
6     2014  2f4d21f8-6d5f-48a5-abca-52a30583871a         Gonzaga   Bulldogs   
7     2015  2f4d21f8-6d5f-48a5-abca-52a30583871a         Gonzaga   Bulldogs   
8     2016  2f4d21f8-6d5f-48a5-abca-52a30583871a         Gonzaga   Bulldogs   
9     2017  2f4d21f8-6d5f-48a5-abca-52a30583871a         Gonzaga   Bulldogs   
10    2013  4383eb6a-7fd8-4ff2-94c5-43c933121e88       Villanova   Wildcats   
11    2014  4383eb6a-7fd8-4ff2-94c5-43c933121e88    

In [184]:
from bokeh.palettes import Category10
from bokeh.models import Legend, LegendItem

p = figure()
r = p.multi_line(xs=xs, ys=ys, color=Category10[5], line_width=2)

teams = topTeamWins.market.unique()
xs = []
ys = []
legendItems = []
index = 0;
for team in teams:
    x = []
    y = []
    for season in [2013, 2014, 2015, 2016, 2017]:
        x.append(season)
        gamesWonSeason = topTeamWins.loc[(topTeamWins['market'] == team) & (topTeamWins['season'] == season)]  
        y.append(gamesWonSeason.iloc[0]['ncaaGamesWon'] if len(gamesWonSeason) == 1 else 0)
    xs.append(x)
    ys.append(y)
    legendItems.append(LegendItem(label=team, renderers=[r], index=index))
    index = index+1

legend=Legend(items=legendItems)
    
p.add_layout(legend)

show(p)